In [28]:
#import relevant libraries 
import pandas as pd
import plotly.express as px
import panel as pn
import hvplot.pandas
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import os
%matplotlib inline
import nltk
nltk.download('vader_lexicon')
from datetime import datetime, timedelta,date
from pandas import DataFrame
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/anna/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [29]:
## Get Covid Case Count Data
#using url for covid 19 api united states case counts confirmed
request_url = "https://api.covid19api.com/total/dayone/country/united-states/status/confirmed"

def get_case_count(request_url):
    # Submit request and format output
    response_data = requests.get(request_url).json()
    states_json = json.dumps(response_data, indent=4)
    case_count_df = pd.read_json(request_url)
    #clean data
    case_count_df.drop(columns = ['CountryCode', 'Province', 'City', 'CityCode', 'Lat','Lon'], inplace = True)
    #update date time stamp to just date
    case_count_df['Date']=case_count_df['Date'].dt.date
    case_count_df.set_index('Date', inplace = True)
    #add percent change column
    case_pct_change = case_count_df['Cases'].pct_change()
    case_count_df['Pct_Change']= case_pct_change
    #add 7 day rolling moving average
    case_rolling = case_count_df['Cases'].rolling(window = 7).mean()
    case_count_df['7day_Rolling'] = case_rolling
    #drop nulls
    case_count_df.dropna(inplace = True)
    return case_count_df

In [30]:
case_count_df=get_case_count(request_url)


In [31]:
## Get Covid 19 News Data
# Read api key environment variable for news api
load_dotenv()
newsapi_key = os.getenv("newsapikey")

import newsapi
from newsapi import NewsApiClient

#get news articles on covid 19
# Create a newsapi client
newsapi = NewsApiClient(api_key=newsapi_key)
    
    

In [34]:
## Get Market Data for S&P500 
#engage API keys by activating .env file for Alpaca Api
load_dotenv()
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

def get_SP500_data(api):
    #load in historical data for S&P 500
    sp500_df = api.alpha_vantage.historic_quotes('SPY', adjusted=True, output_format='pandas')
    
    #Clean Data
    #Select the column we need,  "adjusted close", and drop the others
    sp500_df = sp500_df['5. adjusted close']
    #Sort earliest to latest so that .pct_change() function works right.
    sp500_df.sort_index(inplace=True, ascending=True)

    #create a dataframe column for the daily returns (pct_change) values and concat with SP500 close
    returns_df = sp500_df.pct_change()
    all_returns_df = pd.concat([sp500_df, returns_df], axis="columns", join="inner")

    #Change column names to avoid confusion
    columns = ['S&P 500 close','S&P 500 Daily Returns']
    all_returns_df.columns = columns

    # Drop nulls
    all_returns_df.dropna(inplace=True)

    #drop duplicates
    all_returns_df.drop_duplicates(inplace=True) 
    return all_returns_df

In [39]:
all_returns_df=get_SP500_data(api)

In [19]:
def exp_moving_avg():
    return 

In [20]:
def bolinger_bands():
    return

In [21]:
def covid_case_count(case_count_df):
    case_count_signals_df = case_count_df.loc[:, ['Cases']].copy()
    short_window = 7
    long_window = 30

    case_count_signals_df['7day % Change'] = case_count_signals_df['Cases'].pct_change().rolling(short_window).mean()
    case_count_signals_df['30day % Change'] = case_count_signals_df['Cases'].pct_change().rolling(long_window).mean()

    case_count_signals_df['Case Count Signal'] = 0.0
    case_count_signals_df['Case Count Signal'][short_window:] = np.where(case_count_signals_df['7day % Change'][short_window:] > case_count_signals_df['30day % Change'][short_window:], 1.0, -1.0)

    return case_count_signals_df
    

In [89]:
def covid_sentiment(newsapi):
    daily_signal_dict={}
    daily_sentiment_dict={}

    for i in range(30):

        # Set start and end datetimes of for 1 days of news
        end_date = datetime.now()
        start_date = end_date + timedelta(-i)
        end_date=end_date.strftime("%Y-%m-%d")
        start_date=start_date.strftime("%Y-%m-%d")

        # Fetch the Covid19 news articles
        covid19_news = newsapi.get_everything(
        from_param=start_date,
        to=end_date,
        q="Covid 19",
        language="en",
        page_size=100,
        sort_by="relevancy"
        )
    

        #covid-19 sentiment list to dataframe
        covid19_sentiment_list=[]
        for article in covid19_news["articles"]:        
            try: 
                text = article["content"]
                sentiment = analyzer.polarity_scores(text)
                compound = sentiment["compound"]
                pos = sentiment["pos"]
                neu = sentiment["neu"]
                neg = sentiment["neg"]
                scores={"Compound":compound, "Negative":neg, "Neutral":neu, "Positive":pos, "text":text}
                covid19_sentiment_list.append(scores)
        
            except: 
                pass

        covid19_sentiment_df=pd.DataFrame(covid19_sentiment_list)
    

        # Describe the  Sentiment Related to Covid19/Coronavirus
        sentiment_df=covid19_sentiment_df.describe()


        #determine trading signal value based on covid news sentiment 
        if sentiment_df['Neutral'][1] > (sentiment_df['Positive'][1] and sentiment_df['Negative'][1]):
            sentiment_signal=0.0
            sentiment='Neutral'
        elif sentiment_df['Negative'][1] > (sentiment_df['Positive'][1] and sentiment_df['Neutral'][1]):
            sentiment_signal=-1.0
            sentiment='Negative'
        elif sentiment_df['Positive'][1] > (sentiment_df['Negative'][1] and sentiment_df['Netural'][1]):
            sentiment_signal=1.0
            sentiment='Positive'

        daily_signal_dict.update({start_date:sentiment_signal})
        daily_sentiment_dict.update({start_date:sentiment})
   
    #convert dictionary to dataframe     
    daily_signal_df=DataFrame.from_dict(daily_signal_dict,orient='index',columns=['Sentiment Signal'])
    daily_sentiment_df=DataFrame.from_dict(daily_sentiment_dict,orient='index',columns=['Sentiment'])

    sentiment_signal_df=pd.concat([daily_signal_df,daily_sentiment_df],join='inner', axis=1)
    sentiment_signal_df.sort_index(inplace=True, ascending=True)
    return sentiment_signal_df


In [40]:
case_count_signals_df=covid_case_count(case_count_df)

In [90]:
sentiment_signal_df=covid_sentiment(newsapi)


In [76]:
def get_trading_signals(case_count_signals_df,sentiment_signal_df):#,ema_signal_df,bolinger_signal_df)
    case_count_signals_df=case_count_signals_df.tail(30)
    trading_signals_df=pd.concat([case_count_signals_df, sentiment_signal_df],axis=1)
    trading_signals_df['Signal']=trading_signals_df['Case Count Signal']+trading_signals_df['Sentiment Signal']
    trading_signals_df['Entry/Exit']=trading_signals_df['Signal'].diff()
    return trading_signals_df

In [82]:
trading_signals_df=get_trading_signals(case_count_signals_df,sentiment_signal_df)
trading_signals_df.tail()

,Cases,7day % Change,30day % Change,Case Count Signal,Sentiment Signal,Sentiment,Signal,Entry/Exit
2020-06-23,NaN,NaN,NaN,NaN,0.0,Neutral,NaN,NaN
2020-06-24,NaN,NaN,NaN,NaN,0.0,Neutral,NaN,NaN
2020-06-25,NaN,NaN,NaN,NaN,0.0,Neutral,NaN,NaN
2020-06-26,NaN,NaN,NaN,NaN,0.0,Neutral,NaN,NaN
2020-06-27,NaN,NaN,NaN,NaN,0.0,Neutral,NaN,NaN


In [ ]:
# Initialize Streaming DataFrame for Market Price Data Showing Backtested Signals
    data_stream = Stream()
    data = all_returns_df['S&P 500 close']
    data_stream_df = DataFrame(data_stream, example=data)
# Initialize Streaming DataFrame for Signals
    signals_stream = Stream()
    signals_data=trading_signals_df
    signals_stream_df = DataFrame(signals_stream, example=signals_data)

In [ ]:
##Dashboard
live_column = pn.Column()
backtesting_column = pn.Column()